### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
from sklearn.preprocessing import Normalizer

### Enable Eager Execution if you are using tensorflow 1.x

In [3]:
# Enable Eager Execution
tf.enable_eager_execution()

### Collect Data

In [4]:
import pandas as pd

In [25]:
data = pd.read_csv('prices.csv')

In [26]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Check all columns in the dataset

In [27]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [8]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [28]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [29]:
data = data.drop(columns=['date','symbol'], axis = 1)

In [30]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [31]:
data = data.head(1000)

In [32]:
data.shape

(1000, 5)

In [33]:
data.sample(5)

,open,close,low,high,volume
593,32.389999,33.380001,32.380001,33.720001,712500.0
91,123.120003,123.910004,122.870003,124.610001,528500.0
373,75.480000,75.149998,74.500003,75.680000,5202600.0
991,14.200000,14.420000,14.120000,14.430000,3258700.0
919,3.680000,3.760000,3.650000,3.770000,21226600.0


### Convert Float64 to Float32

In [34]:
data = data.astype('float32')

### Divide the data into train and test sets

In [35]:
from sklearn.model_selection import train_test_split

X1 = data.drop(columns= "close" , axis = 1) 
Y1 = data["close"]


### Normalize Train and Test Data 

In [36]:


train_x, test_x,train_y,test_y = train_test_split(X1, Y1, test_size=.3, random_state=7)

In [37]:
import numpy as np
transformer = Normalizer()
train_X= transformer.fit_transform(train_x)
train_Y= np.array(train_y)

## Building the graph in tensorflow

2.Define Weights and Bias

In [38]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [39]:
def prediction(x, w, b):
   
   xw_matmul = tf.matmul(x, w)
   y = tf.add(xw_matmul, b)
   
   return y

4.Loss (Cost) Function [Mean square error]

In [40]:
def loss(y_actual, y_predicted):
   
   diff = y_actual - y_predicted
   sqr = tf.square(diff)
   avg = tf.reduce_mean(sqr)
   
   return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [41]:
def train(x, y_actual, w, b, learning_rate=0.01):
   
   #Record mathematical operations on 'tape' to calculate loss
   with tf.GradientTape() as t:
       
       t.watch([w,b])
       
       current_prediction = prediction(x, w, b)
       current_loss = loss(y_actual, current_prediction)
   
   #Calculate Gradients for Loss with respect to Weights and Bias
   dw, db = t.gradient(current_loss,[w, b])
   
   #Update Weights and Bias
   w = w - learning_rate*dw
   b = b - learning_rate*db
   
   return w, b

## Execute the Graph for 100 epochs and observe the loss

In [42]:
for i in range(100):
   
   w, b = train(train_X, train_Y, w, b)
   print('Current Loss on iteration', i, loss(train_Y, prediction(train_X, w, b)).numpy())

Current Loss on iteration 0 8002.6514
Current Loss on iteration 1 7663.568
Current Loss on iteration 2 7351.0693
Current Loss on iteration 3 7063.065
Current Loss on iteration 4 6797.6504
Current Loss on iteration 5 6553.033
Current Loss on iteration 6 6327.599
Current Loss on iteration 7 6119.838
Current Loss on iteration 8 5928.3647
Current Loss on iteration 9 5751.9062
Current Loss on iteration 10 5589.279
Current Loss on iteration 11 5439.4014
Current Loss on iteration 12 5301.2764
Current Loss on iteration 13 5173.9775
Current Loss on iteration 14 5056.66
Current Loss on iteration 15 4948.547
Current Loss on iteration 16 4848.9023
Current Loss on iteration 17 4757.0684
Current Loss on iteration 18 4672.4365
Current Loss on iteration 19 4594.4424
Current Loss on iteration 20 4522.562
Current Loss on iteration 21 4456.313
Current Loss on iteration 22 4395.265
Current Loss on iteration 23 4338.999
Current Loss on iteration 24 4287.1455
Current Loss on iteration 25 4239.3525
Current L

### Get the shapes and values of W and b

In [43]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5353243e-03]
 [2.5108384e-03]
 [2.5564781e-03]
 [3.3674854e+01]]
Bias:
 [33.67486]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [44]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


### Execute the model

In [45]:
model.fit(train_x,train_y, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
700/700 [==============================] - 1s 1ms/sample - loss: nan          
Epoch 2/100
700/700 [==============================] - 0s 82us/sample - loss: nan
Epoch 3/100
700/700 [==============================] - 0s 71us/sample - loss: nan
Epoch 4/100
700/700 [==============================] - 0s 57us/sample - loss: nan
Epoch 5/100
700/700 [==============================] - 0s 120us/sample - loss: nan
Epoch 6/100
700/700 [==============================] - 0s 108us/sample - loss: nan
Epoch 7/100
700/700 [==============================] - 0s 120us/sample - loss: nan
Epoch 8/100
700/700 [==============================] - 0s 90us/sample - loss: nan
Epoch 9/100
700/700 [==============================] - 0s 91us/sample - loss: nan
Epoch 10/100
700/700 [==============================] - 0s 63us/sample - loss: nan
Epoch 11/100
700/700 [==============================] - 0s 72us/sample - loss: nan
Epoch 12/100
700/700 [==============

700/700 [==============================] - 0s 46us/sample - loss: nan
Epoch 97/100
700/700 [==============================] - 0s 51us/sample - loss: nan
Epoch 98/100
700/700 [==============================] - 0s 46us/sample - loss: nan
Epoch 99/100
700/700 [==============================] - 0s 57us/sample - loss: nan
Epoch 100/100
700/700 [==============================] - 0s 46us/sample - loss: nan


In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.get_weights()

[array([[nan],
        [nan],
        [nan],
        [nan]], dtype=float32), array([nan], dtype=float32)]

In [51]:
from sklearn.model_selection import train_test_split


transformer = Normalizer()
X1= transformer.fit_transform(X1)
Y1= transformer.fit_transform(Y1)
train_x, test_x,train_y,test_y = train_test_split(X1, Y1, test_size=.3, random_state=7)

ValueError: Expected 2D array, got 1D array instead:
array=[125.84     119.98     114.95     116.62     114.97     115.55
 112.85     114.38     112.53     110.38     109.3      110.
 111.95     110.12     111.       110.71     112.58     114.47
 114.5      110.56     114.05     115.71     114.02     111.16
 110.65     107.52     107.13     107.84     110.77     111.24
 111.6      110.33     113.04     111.89     111.56     112.88
 112.75     113.32     115.51     116.78     117.       117.19
 116.95     116.71     116.49     116.82     120.62     120.63
 120.7      120.82     124.03     121.27     121.45     121.47
 119.38     119.41     118.72     117.98     118.74     118.66
 119.93     117.65     114.07     115.47     113.54     113.41
 114.64     115.79     117.99     117.72     118.45     121.26
 121.68     122.31     123.01     123.79     123.3      124.4
 124.69     124.15     124.9      125.96     126.27     126.56
 127.66     121.56     123.44     125.51     124.21     123.89

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [52]:
# iris= mount_drive("Iris-1.csv")
iris = pd.read_csv('Iris-2.csv')

In [53]:
iris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [54]:
iris.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [55]:
iris["Species"] = pd.Categorical(iris["Species"]).codes

In [56]:
iris.drop("Id",axis=1,inplace=True)

### Splitting the data into feature set and target set

In [57]:
features = iris.drop(columns= ['Species']) 

target = iris[['Species']]

In [58]:

features.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [59]:
target.head(5)

,Species
0,0
1,0
2,0
3,0
4,0


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [60]:
from sklearn.preprocessing import LabelEncoder

In [61]:
Y_processed = pd.DataFrame(target).apply(LabelEncoder().fit_transform)

In [62]:
testY = tf.keras.utils.to_categorical(Y_processed, num_classes=3)

In [63]:
testY

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

### Divide the dataset into Training and test (70:30)

In [64]:
from sklearn.model_selection import train_test_split
transformer = Normalizer()
#features = transformer.fit_transform(features)

train_x, test_x,train_y,test_y = train_test_split(features, testY, test_size=.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [65]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [66]:
#train model
model.fit(train_x.values, train_y, validation_split=0.2, epochs=30)

Train on 84 samples, validate on 21 samples
Epoch 1/30
84/84 [==============================] - 2s 24ms/sample - loss: 1.0351 - acc: 0.5833 - val_loss: 1.2510 - val_acc: 0.3333
Epoch 2/30
84/84 [==============================] - 0s 324us/sample - loss: 1.0221 - acc: 0.5714 - val_loss: 1.2001 - val_acc: 0.3333
Epoch 3/30
84/84 [==============================] - 0s 238us/sample - loss: 0.9958 - acc: 0.6429 - val_loss: 1.1528 - val_acc: 0.3333
Epoch 4/30
84/84 [==============================] - 0s 190us/sample - loss: 0.9724 - acc: 0.7500 - val_loss: 1.1098 - val_acc: 0.3333
Epoch 5/30
84/84 [==============================] - 0s 190us/sample - loss: 0.9540 - acc: 0.7857 - val_loss: 1.0666 - val_acc: 0.3810
Epoch 6/30
84/84 [==============================] - 0s 143us/sample - loss: 0.9359 - acc: 0.7024 - val_loss: 1.0337 - val_acc: 0.4762
Epoch 7/30
84/84 [==============================] - 0s 143us/sample - loss: 0.9228 - acc: 0.6667 - val_loss: 1.0024 - val_acc: 0.6190
Epoch 8/30
84/84 [=

### Report Accuracy of the predicted values

In [67]:
model.evaluate(test_x,test_y)

45/45 [==============================] - 0s 1ms/sample - loss: 0.7739 - acc: 0.6444


[0.7739126867718167, 0.64444447]

In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1 (Batc multiple                  16        
_________________________________________________________________
dense_1 (Dense)              multiple                  50        
_________________________________________________________________
dense_2 (Dense)              multiple                  88        
_________________________________________________________________
dense_3 (Dense)              multiple                  27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________
